# MVP-V: KITTI 데이터 로딩

이 노트북은 KITTI 데이터셋의 이미지와 레이블을 로딩하고 시각화합니다.

## 목표
- KITTI 이미지 데이터 로딩 (left/right 스테레오)
- 레이블 파싱 (2D 바운딩 박스)
- 칼리브레이션 정보 로딩
- 데이터 시각화 (이미지 + 바운딩 박스 오버레이)

In [ ]:
# Cell 1: 라이브러리 설치 및 임포트
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import cv2

# 경로 설정
project_root = Path.cwd().parent.parent
dataset_root = project_root / "dataset" / "kitti"
print(f"프로젝트 루트: {project_root}")
print(f"데이터셋 루트: {dataset_root}")

In [ ]:
# Cell 2: KITTI 데이터셋 구조 확인
def check_dataset_structure(dataset_root):
    """KITTI 데이터셋 구조 확인"""
    if not dataset_root.exists():
        print(f"경고: 데이터셋 경로가 존재하지 않습니다: {dataset_root}")
        return False
    
    print("데이터셋 구조:")
    for item in sorted(dataset_root.iterdir()):
        if item.is_dir():
            file_count = len(list(item.rglob("*")))
            print(f"  {item.name}/ ({file_count} files)")
        else:
            print(f"  {item.name}")
    
    return True

check_dataset_structure(dataset_root)

In [ ]:
# Cell 3: KITTI 레이블 파싱 함수
def parse_kitti_label(label_path):
    """
    KITTI 레이블 파일 파싱
    형식: class truncated occluded alpha bbox_2d dimensions_3d location_3d rotation_y
    """
    annotations = []
    if not label_path.exists():
        return annotations
    
    with open(label_path, 'r') as f:
        for line in f:
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 15:
                    annotation = {
                        'class': parts[0],
                        'truncated': float(parts[1]),
                        'occluded': int(parts[2]),
                        'alpha': float(parts[3]),
                        'bbox_2d': [float(parts[4]), float(parts[5]), 
                                   float(parts[6]), float(parts[7])],  # x1, y1, x2, y2
                        'dimensions_3d': [float(parts[8]), float(parts[9]), float(parts[10])],  # h, w, l
                        'location_3d': [float(parts[11]), float(parts[12]), float(parts[13])],  # x, y, z
                        'rotation_y': float(parts[14])
                    }
                    annotations.append(annotation)
    
    return annotations

# 테스트
train_labels_dir = dataset_root / "labels" / "train"
if train_labels_dir.exists():
    sample_label = list(train_labels_dir.glob("*.txt"))[0] if list(train_labels_dir.glob("*.txt")) else None
    if sample_label:
        print(f"샘플 레이블: {sample_label.name}")
        annotations = parse_kitti_label(sample_label)
        print(f"탐지된 객체 수: {len(annotations)}")
        for i, ann in enumerate(annotations[:3]):
            print(f"  객체 {i+1}: {ann['class']}, bbox: {ann['bbox_2d']}")

In [ ]:
# Cell 4: 칼리브레이션 정보 로딩 함수
def load_calibration(calib_path):
    """
    KITTI 칼리브레이션 파일 로딩
    주로 training 폴더에 calib 폴더가 있음
    """
    calib = {}
    if not calib_path.exists():
        print(f"경고: 칼리브레이션 파일이 없습니다: {calib_path}")
        return calib
    
    with open(calib_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                calib[key.strip()] = np.array([float(x) for x in value.strip().split()]).reshape(3, -1)
    
    return calib

# 칼리브레이션 경로 확인 (일반적으로 training/calib 또는 별도 폴더)
training_calib = project_root / "dataset" / "training" / "calib"
if training_calib.exists():
    sample_calib = list(training_calib.glob("*.txt"))[0] if list(training_calib.glob("*.txt")) else None
    if sample_calib:
        print(f"샘플 칼리브레이션: {sample_calib.name}")
        calib = load_calibration(sample_calib)
        print(f"칼리브레이션 키: {list(calib.keys())}")
        if 'P2' in calib:
            print(f"P2 (왼쪽 카메라):\n{calib['P2']}")

In [ ]:
# Cell 5: 이미지와 레이블 로딩 함수
def load_kitti_sample(image_dir, label_dir, index, split='train'):
    """
    KITTI 샘플 로딩 (이미지 + 레이블)
    """
    image_path = image_dir / f"{index:06d}.png"
    label_path = label_dir / f"{index:06d}.txt"
    
    if not image_path.exists():
        raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {image_path}")
    
    # 이미지 로딩
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 레이블 로딩
    annotations = parse_kitti_label(label_path)
    
    return image, annotations

# 테스트: 첫 번째 샘플 로딩
train_images_dir = dataset_root / "images" / "train"
train_labels_dir = dataset_root / "labels" / "train"

if train_images_dir.exists() and train_labels_dir.exists():
    image_files = sorted(train_images_dir.glob("*.png"))
    if image_files:
        sample_idx = 0
        image, annotations = load_kitti_sample(train_images_dir, train_labels_dir, sample_idx)
        print(f"로딩된 이미지 shape: {image.shape}")
        print(f"레이블 개수: {len(annotations)}")
    else:
        print("훈련 이미지 파일을 찾을 수 없습니다.")
else:
    print("훈련 데이터 디렉토리를 찾을 수 없습니다.")

In [ ]:
# Cell 6: 데이터 시각화 함수
def visualize_kitti_sample(image, annotations, title="KITTI Sample"):
    """
    KITTI 샘플 시각화 (이미지 + 바운딩 박스)
    """
    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.imshow(image)
    ax.set_title(title, fontsize=14)
    ax.axis('off')
    
    # 클래스별 색상 정의
    class_colors = {
        'Car': 'red',
        'Van': 'orange',
        'Truck': 'yellow',
        'Pedestrian': 'green',
        'Person_sitting': 'lightgreen',
        'Cyclist': 'blue',
        'Tram': 'purple',
        'Misc': 'gray'
    }
    
    for ann in annotations:
        class_name = ann['class']
        bbox = ann['bbox_2d']
        color = class_colors.get(class_name, 'white')
        
        # 바운딩 박스 그리기
        x1, y1, x2, y2 = bbox
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           fill=False, edgecolor=color, linewidth=2)
        ax.add_patch(rect)
        
        # 클래스 이름 표시
        ax.text(x1, y1-5, f"{class_name}", 
               color=color, fontsize=10, weight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.5))
    
    plt.tight_layout()
    return fig

# 시각화 테스트
if 'image' in locals() and 'annotations' in locals():
    fig = visualize_kitti_sample(image, annotations, f"KITTI Sample {sample_idx:06d}")
    plt.show()

In [ ]:
# Cell 7: 데이터셋 통계 분석
def analyze_dataset(image_dir, label_dir, split='train'):
    """
    데이터셋 통계 분석
    """
    image_files = sorted(image_dir.glob("*.png"))
    label_files = sorted(label_dir.glob("*.txt"))
    
    print(f"\n=== {split.upper()} 데이터셋 통계 ===")
    print(f"이미지 개수: {len(image_files)}")
    print(f"레이블 개수: {len(label_files)}")
    
    # 클래스별 통계
    class_counts = {}
    total_objects = 0
    
    for label_file in label_files:
        annotations = parse_kitti_label(label_file)
        for ann in annotations:
            class_name = ann['class']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
            total_objects += 1
    
    print(f"\n총 객체 개수: {total_objects}")
    print("\n클래스별 분포:")
    for class_name, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_objects * 100) if total_objects > 0 else 0
        print(f"  {class_name:15s}: {count:5d} ({percentage:5.2f}%)")
    
    return class_counts

# 통계 분석 실행
if train_images_dir.exists() and train_labels_dir.exists():
    train_stats = analyze_dataset(train_images_dir, train_labels_dir, 'train')